<a href="https://colab.research.google.com/github/Choiyh1116/AI_capstone/blob/main/Code/VideoCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import cv2
import os
import numpy as np
import matplotlib.pylab as plt

vidcap = cv2.VideoCapture('테스트1.mp4')
methods = ['CORREL', 'CHISQR', 'INTERSECT', 'BHATTACHARYYA', 'EMD']
hists = []

#csv파일관련
cor, chi, ist, bha, emd, sct = list(), list(), list(), list(), list(), list()
csvName = '테스트1.csv'
filecsv = open(csvName, "w")

#영상 못불러오면 코드종료
if not vidcap.isOpened():
    print("Could not Open")
    exit(0)

count = 0

#각각 초당프레임, 영상 총 프레임 수, 계산된 영상 길이(초단위)
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
totalf = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
totalt = totalf/fps
#print("FPS : %d\t영상길이 : %f"%(fps,totalt))

#코드돌리는시간이 너무 길다면, 조절한다 -> frameGap
frameGap = int(fps/30)
gapCount = int(frameGap-1)

while(vidcap.isOpened()):
    if(gapCount==0):
        ret, image = vidcap.read()
        gapCount = int(frameGap-1)
    else:
        ret = vidcap.grab()
        gapCount -= 1
        continue
    if ret:
        if(int (vidcap.get(1) % frameGap == 0)):
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
            cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
            hists.append(hist)
                
            if(len(hists)==2):
                query = hists[0]
                for index, name in enumerate(methods):
    
                    for i, histogram in enumerate(hists):
                        if(i == 0):
                            continue
                        compare = cv2.compareHist(query, histogram, index) 
        
                        if(index == cv2.HISTCMP_INTERSECT):
                            compare = compare/np.sum(query)
                        
                        if(name == 'CORREL'):
                            cor.append(compare)
                        if(name == 'CHISQR'):
                            chi.append(compare)
                        if(name == 'INTERSECT'):
                            ist.append(compare)
                        if(name == 'BHATTACHARYYA'):
                            bha.append(compare)
                        if(name == 'EMD'):
                            emd.append(compare)
                            sct.append(0)
                            
                del(hists[0])
            
            
            count += 1
    else:
        break
        
filecsv.write('CORREL,CHISQR,INTERSECT,BHATTACHARYYA,EMD,Screen_Transition\n')
filecsv.write(str(fps)+','+str(totalf)+','+str(frameGap)+',0,0,0\n')
for i in range(len(cor)):
    filecsv.write(str(cor[i]) + ',' + str(chi[i]) + ',' + str(ist[i]) + ',' + str(bha[i]) + ',' + str(emd[i]) + ',' + str(sct[i]) + '\n')
    
filecsv.close()

del(cor,chi,ist,bha,emd,sct)

vidcap.release()

In [9]:
import pickle
import joblib
import pandas as pd

df = pd.read_csv(csvName)
dl = df.loc[0].to_list()
df.drop(0,axis=0,inplace=True)

fps, totalf, frameGap = dl[0], dl[1], dl[2]

r1 = ['CORREL','CHISQR','INTERSECT','BHATTACHARYYA','EMD']
#r2 = ['Screen_Transition']

x = df[r1]
#y = df['Screen_Transition']

model = joblib.load('model.pkl')

pred = model.predict(x)

predDf = pd.DataFrame(pred)
predDf.columns = ['Screen_Transition']

predList = predDf.loc[predDf['Screen_Transition']==1].index.tolist()

acttime = []
for i in range(len(predList)):
    sec = predList[i]*frameGap/fps
    minute = int(sec // 60)
    sec = sec - (minute*60)
    acttime.append(str(minute)+'분 '+str(sec)+'초')

print("총 장면전환 개수 : %d", len(predList))

for i in range(len(acttime)):
    print('\t\t'+acttime[i])

x = pd.concat([x,predDf],axis=1)

x.to_csv(csvName, mode='w')

총 장면전환 개수 : %d 67
		0분 0.4666666666666667초
		0분 4.866666666666666초
		0분 6.033333333333333초
		0분 7.366666666666666초
		0분 8.633333333333333초
		0분 8.666666666666666초
		0분 9.833333333333334초
		0분 9.866666666666667초
		0분 23.4초
		0분 23.433333333333334초
		0분 26.533333333333335초
		0분 31.833333333333332초
		0분 31.866666666666667초
		0분 33.666666666666664초
		0분 34.43333333333333초
		0분 35.93333333333333초
		0분 35.96666666666667초
		0분 49.06666666666667초
		0분 51.2초
		0분 51.233333333333334초
		1분 0.7666666666666657초
		1분 2.3333333333333357초
		1분 4.333333333333329초
		1분 4.36666666666666초
		1분 6.5초
		1분 6.566666666666663초
		1분 6.63333333333334초
		1분 7.933333333333337초
		1분 7.966666666666669초
		1분 9.299999999999997초
		1분 9.36666666666666초
		1분 11.799999999999997초
		1분 13.433333333333337초
		1분 15.666666666666671초
		1분 15.700000000000003초
		1분 23.5초
		1분 25.266666666666666초
		1분 34.900000000000006초
		1분 35.099999999999994초
		1분 36.63333333333334초
		1분 36.7초
		1분 40.900000000000006초
		1분 40.93333333333334초
		